In [ ]:
##!/usr/bin/env python
"""plot_skill.py

Script plots the normalised error scores for WRF-LIS-CABLE physics ensemble

Author: Annette L Hirsch @ CLEX, UNSW. Sydney (Australia)
email: a.hirsch@unsw.edu.au
Created: Thu May  2 12:15:30 AEST 2019

"""

In [1]:
# Load packages

#from __future__ import division
import numpy as np
import netCDF4 as nc
import sys
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
import xarray
from mpl_toolkits.basemap import Basemap
import common_functions as cf


/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.01/lib/python3.6/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.01/lib/python3.6/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [2]:
sims=["PHYS_TEST_RA1_PBL1_CU1", "PHYS_TEST_RA1_PBL1_CU16", "PHYS_TEST_RA1_PBL1_CU2", "PHYS_TEST_RA1_PBL1_CU3", 
      "PHYS_TEST_RA4_PBL1_CU1", "PHYS_TEST_RA4_PBL1_CU16", "PHYS_TEST_RA4_PBL1_CU2", "PHYS_TEST_RA4_PBL1_CU3", 
      "PHYS_TEST_RA5_PBL1_CU1", "PHYS_TEST_RA5_PBL1_CU16", "PHYS_TEST_RA5_PBL1_CU2", "PHYS_TEST_RA5_PBL1_CU3", 
      "PHYS_TEST_RA1_PBL2_CU1", "PHYS_TEST_RA1_PBL2_CU16", "PHYS_TEST_RA1_PBL2_CU2", "PHYS_TEST_RA1_PBL2_CU3", 
#      "PHYS_TEST_RA3_PBL2_CU1", "PHYS_TEST_RA3_PBL2_CU16", "PHYS_TEST_RA3_PBL2_CU2", 
      "PHYS_TEST_RA4_PBL2_CU1", "PHYS_TEST_RA4_PBL2_CU16", "PHYS_TEST_RA4_PBL2_CU2", "PHYS_TEST_RA4_PBL2_CU3", 
      "PHYS_TEST_RA5_PBL2_CU1", "PHYS_TEST_RA5_PBL2_CU16", "PHYS_TEST_RA5_PBL2_CU2", "PHYS_TEST_RA5_PBL2_CU3", 
      "PHYS_TEST_RA1_PBL5_CU1", "PHYS_TEST_RA1_PBL5_CU16", "PHYS_TEST_RA1_PBL5_CU2", "PHYS_TEST_RA1_PBL5_CU3", 
      "PHYS_TEST_RA4_PBL5_CU1", "PHYS_TEST_RA4_PBL5_CU16", "PHYS_TEST_RA4_PBL5_CU2", "PHYS_TEST_RA4_PBL5_CU3", 
      "PHYS_TEST_RA5_PBL5_CU1", "PHYS_TEST_RA5_PBL5_CU16", "PHYS_TEST_RA5_PBL5_CU2", "PHYS_TEST_RA5_PBL5_CU3"]
rlabels=["Dudhia / YSU / KF","Dudhia / YSU / NTiedtke","Dudhia / YSU / BMJ","Dudhia / YSU / GF",
         "RRTMG / YSU / KF","RRTMG / YSU / NTiedtke","RRTMG / YSU / BMJ","RRTMG / YSU / GF",
         "NGoddard / YSU / KF","NGoddard / YSU / NTiedtke","NGoddard / YSU / BMJ","NGoddard / YSU / GF",
         "Dudhia / MYJ / KF","Dudhia / MYJ / NTiedtke","Dudhia / MYJ / BMJ","Dudhia / MYJ / GF",
#         "MYJ / CAM / KF","MYJ / CAM / NTiedtke","MYJ / CAM / BMJ",
         "RRTMG / MYJ / KF","RRTMG / MYJ / NTiedtke","RRTMG / MYJ / BMJ","RRTMG / MYJ / GF",
         "NGoddard / MYJ / KF","NGoddard / MYJ / NTiedtke","NGoddard / MYJ / BMJ","NGoddard / MYJ / GF",
         "Dudhia / MYNN / KF","Dudhia / MYNN / NTiedtke","Dudhia / MYNN / BMJ","Dudhia / MYNN / GF",
         "RRTMG / MYNN / KF","RRTMG / MYNN / NTiedtke","RRTMG / MYNN / BMJ","RRTMG / MYNN / GF",
         "NGoddard / MYNN / KF","NGoddard / MYNN / NTiedtke","NGoddard / MYNN / BMJ","NGoddard / MYNN / GF"]
group="PBL"
fig_name_prefix='SCALAR_SCORE_NO_CAM_AWAP_MASKED'


In [3]:
# Simulation Details - grouped by PBL

sims=["PHYS_TEST_RA5_PBL5_CU2", "SOILSNOW_RA5_PBL5_CU2"]
rlabels=["SSGW","SOILSNOW"]
fig_name_prefix='HYDRO_AWAP_MASKED'


In [2]:
# Simulation Details - grouped by PBL

sims=["PHYS_TEST_RA4_PBL2_CU16", "PHYS_TEST_8YR","PHYS_30YR_SPIN"]
rlabels=["4YR","8YR", "30YR"]
fig_name_prefix='SPIN_SKILL_AWAP_MASKED'


In [3]:
# Define USER input arguments

# Year / Period of interest
syear = np.int('2008')
eyear = np.int('2010')

# Data directory 
datadir = '/g/data/hh5/tmp/WRF-CABLE/AUS44/postprocess/'

nmod = len(sims)
obs = "AWAP"

# Figure Details
fig_dir='%s/figures/' %(os.getcwd())
#fig_name_prefix='SCALAR_SCORE'
if not os.path.exists(fig_dir):
  os.makedirs(fig_dir)

# Landsea mask
mask_file='/g/data/hh5/tmp/WRF-CABLE/AUS44/PHYS_TEST_MASTER/geo_em.d01.nc'
lis_file='/g/data/hh5/tmp/WRF-CABLE/AUS44/PHYS_TEST_MASTER/bdy_data/lis_input.d01.nc'

# Variables
variables=['PR','TX','TN','QE']
nvar=len(variables)

# Define regions to zoom in on
latN=[-23.43,-38.0,-34.0]
latX=[-10.0,-27.0,-30.0]
lonN=[110.0,139.0,114.0]
lonX=[155.0,151.0,120.0]
Rname=["TROPICS","SEA","SWWA"]
nreg = len(Rname)

# Define the mean climate intervals of interest    
mclim=["ANN","DJF","MAM","JJA","SON"]
ytarget=["2009"]
mtarget=[["12","01","02"],["03","04","05"],["06","07","08"],["09","10","11"]]
nmean = len(mclim)

# Define the extremes of interest
extremes=["TX05","TX95","TXx","TN05","TN95","TNn","DTR","RX1DAY","RX5DAY","R10mm","CDD"]
nex = len(extremes)


vlabels=['$PR_{ANN}$','$PR_{DJF}$','$PR_{MAM}$','$PR_{JJA}$','$PR_{SON}$',
         '$TX_{ANN}$','$TX_{DJF}$','$TX_{MAM}$','$TX_{JJA}$','$TX_{SON}$',
         '$TN_{ANN}$','$TN_{DJF}$','$TN_{MAM}$','$TN_{JJA}$','$TN_{SON}$',
         '$QE_{ANN}$','$QE_{DJF}$','$QE_{MAM}$','$QE_{JJA}$','$QE_{SON}$',
         '$PR_{ANN}$','$PR_{DJF}$','$PR_{MAM}$','$PR_{JJA}$','$PR_{SON}$',
         '$TX_{ANN}$','$TX_{DJF}$','$TX_{MAM}$','$TX_{JJA}$','$TX_{SON}$',
         '$TN_{ANN}$','$TN_{DJF}$','$TN_{MAM}$','$TN_{JJA}$','$TN_{SON}$',
         '$QE_{ANN}$','$QE_{DJF}$','$QE_{MAM}$','$QE_{JJA}$','$QE_{SON}$',
         '$PR_{ANN}$','$PR_{DJF}$','$PR_{MAM}$','$PR_{JJA}$','$PR_{SON}$',
         '$TX_{ANN}$','$TX_{DJF}$','$TX_{MAM}$','$TX_{JJA}$','$TX_{SON}$',
         '$TN_{ANN}$','$TN_{DJF}$','$TN_{MAM}$','$TN_{JJA}$','$TN_{SON}$',
         '$QE_{ANN}$','$QE_{DJF}$','$QE_{MAM}$','$QE_{JJA}$','$QE_{SON}$']

elabels=["$TX_{5}$","$TX_{95}$","TXx","$TN_{5}$","$TN_{95}$","TNn","DTR","$RX_{1DAY}$","$RX_{5DAY}$","$R_{10mm}$","CDD",
        "$TX_{5}$","$TX_{95}$","TXx","$TN_{5}$","$TN_{95}$","TNn","DTR","$RX_{1DAY}$","$RX_{5DAY}$","$R_{10mm}$","CDD",
        "$TX_{5}$","$TX_{95}$","TXx","$TN_{5}$","$TN_{95}$","TNn","DTR","$RX_{1DAY}$","$RX_{5DAY}$","$R_{10mm}$","CDD"]


In [4]:
# Load the mask file
filemask    = nc.Dataset(lis_file)
lsmask      = filemask.variables['LANDMASK'][:]
lsmask      = lsmask.astype(int)  
lat2d       = filemask.variables['lat'][:]
lon2d       = filemask.variables['lon'][:]
filemask.close()

In [5]:
# Load observational data

filename='%s/%s_regrid_daily_%s_%s_mask.nc' %(datadir,obs,syear,eyear)
file = nc.Dataset(filename)
obsprecip = file.variables['awap_precip_mask'][:]
obst2max = file.variables['awap_t2max_mask'][:]
obst2min = file.variables['awap_t2min_mask'][:]
file.close()

filename='%s/GLEAM_regrid_daily_%s_%s.nc' %(datadir,syear,eyear)
file = nc.Dataset(filename)
obshfls = file.variables['gleam_hfls_mask'][:]
file.close()

ndays = 822-61
sind = 213+61

# Truncate data to simulation period - removing the spinup aswell
obsprecipT = obsprecip[sind:sind+ndays,:,:]
obst2maxT = obst2max[sind:sind+ndays,:,:]
obst2minT = obst2min[sind:sind+ndays,:,:]
obshflsT = obshfls[sind:sind+ndays,:,:]

# Mask missing values
obsprecipma = np.ma.masked_array(obsprecipT, obsprecipT>=1.e20).filled(np.nan)
obst2maxma  = np.ma.masked_array(obst2maxT, obst2maxT>=1.e20).filled(np.nan)
obst2minma  = np.ma.masked_array(obst2minT, obst2minT>=1.e20).filled(np.nan)
obshflsma  = np.ma.masked_array(obshflsT, obst2maxT>=1.e20).filled(np.nan)


In [6]:
sbias_mean = np.empty((nmod,nreg,nmean,nvar),dtype=np.float64)
srmse_mean = np.empty((nmod,nreg,nmean,nvar),dtype=np.float64)
sphase_mean = np.empty((nmod,nreg,nmean,nvar),dtype=np.float64)
sdsn_mean = np.empty((nmod,nreg,nmean,nvar),dtype=np.float64)
sbias_extr = np.empty((nmod,nreg,nex),dtype=np.float64)
sdsn_extr = np.empty((nmod,nreg,nex),dtype=np.float64)

# Loop through the experiments
for mind,mname in enumerate(sims):

    # Load precip data - excluding 2 month spinup
    filename='%s/%s_WRF_Precip_daily_%s_%s.nc' %(datadir,mname,syear,eyear)
    file = nc.Dataset(filename)
    precip = file.variables['wrf_precip_mask'][61:,:,:]
    file.close()

    # Load temperature data - excluding 2 month spinup
    filename='%s/%s_WRFext_Temps_daily_%s_%s.nc' %(datadir,mname,syear,eyear)
    file = nc.Dataset(filename)
    t2max = file.variables['T2MAX'][61:,:,:]
    t2min = file.variables['T2MIN'][61:,:,:]
    time = nc.chartostring(file.variables['Times'][61:,:])
    timesplit = np.asarray([time[x].split("_")[0] for x in range(len(time))])
    years = np.asarray([timesplit[x].split("-")[0] for x in range(len(time))])
    months = np.asarray([timesplit[x].split("-")[1] for x in range(len(time))])
    file.close()

    # Load hfls data - excluding 2 month spinup
    filename='%s/%s_lisout_daily_%s_%s.nc' %(datadir,mname,syear,eyear)
    file = nc.Dataset(filename)
    hfls = file.variables['lis_hfls_mask'][61:,:,:]
    lat2d = file.variables['wrf_lat2d'][:]
    lon2d = file.variables['wrf_lon2d'][:]
    file.close()
    
    # Loop through the regions
    for rind,rname in enumerate(Rname):
        
        # Get lat/lon indices to truncate data to region of interest
        bbox = [lonN[rind],lonX[rind], latN[rind], latX[rind]]
        i0,i1,j0,j1 = cf.bbox2ij(lon2d,lat2d,bbox)
        
        # Truncate to the period of interest for climate means
        for pind,pname in enumerate(mclim):
            
            # Get relevant time indices
            if pname in ["ANN"]:
                tind = np.where(years==ytarget)[0]
            else:
                tind = np.asarray([months[x] in mtarget[pind-1] for x in range(len(months))])

            # Calculate the bias score
            sbias_mean[mind,rind,pind,0] = cf.calc_bias(precip[tind,j0:j1,i0:i1],obsprecipma[tind,j0:j1,i0:i1],lat2d[j0:j1,i0:i1])
            sbias_mean[mind,rind,pind,1] = cf.calc_bias(t2max[tind,j0:j1,i0:i1],obst2maxma[tind,j0:j1,i0:i1],lat2d[j0:j1,i0:i1])
            sbias_mean[mind,rind,pind,2] = cf.calc_bias(t2min[tind,j0:j1,i0:i1],obst2minma[tind,j0:j1,i0:i1],lat2d[j0:j1,i0:i1])
            sbias_mean[mind,rind,pind,3] = cf.calc_bias(hfls[tind,j0:j1,i0:i1],obshflsma[tind,j0:j1,i0:i1],lat2d[j0:j1,i0:i1])

            # Calculate the rmse score
            srmse_mean[mind,rind,pind,0] = cf.calc_rmse(precip[tind,j0:j1,i0:i1],obsprecipma[tind,j0:j1,i0:i1],lat2d[j0:j1,i0:i1])
            srmse_mean[mind,rind,pind,1] = cf.calc_rmse(t2max[tind,j0:j1,i0:i1],obst2maxma[tind,j0:j1,i0:i1],lat2d[j0:j1,i0:i1])
            srmse_mean[mind,rind,pind,2] = cf.calc_rmse(t2min[tind,j0:j1,i0:i1],obst2minma[tind,j0:j1,i0:i1],lat2d[j0:j1,i0:i1])
            srmse_mean[mind,rind,pind,3] = cf.calc_rmse(hfls[tind,j0:j1,i0:i1],obshflsma[tind,j0:j1,i0:i1],lat2d[j0:j1,i0:i1])

            # Calculate the phase shift score
            sphase_mean[mind,rind,pind,0] = cf.calc_phase(precip[tind,j0:j1,i0:i1],obsprecipma[tind,j0:j1,i0:i1],lat2d[j0:j1,i0:i1])
            sphase_mean[mind,rind,pind,1] = cf.calc_phase(t2max[tind,j0:j1,i0:i1],obst2maxma[tind,j0:j1,i0:i1],lat2d[j0:j1,i0:i1])
            sphase_mean[mind,rind,pind,2] = cf.calc_phase(t2min[tind,j0:j1,i0:i1],obst2minma[tind,j0:j1,i0:i1],lat2d[j0:j1,i0:i1])
            sphase_mean[mind,rind,pind,3] = cf.calc_phase(hfls[tind,j0:j1,i0:i1],obshflsma[tind,j0:j1,i0:i1],lat2d[j0:j1,i0:i1])

            # Calculate the spatial dsn score
            sdsn_mean[mind,rind,pind,0] = cf.calc_spatialdsn(precip[tind,j0:j1,i0:i1],obsprecipma[tind,j0:j1,i0:i1],lat2d[j0:j1,i0:i1])
            sdsn_mean[mind,rind,pind,1] = cf.calc_spatialdsn(t2max[tind,j0:j1,i0:i1],obst2maxma[tind,j0:j1,i0:i1],lat2d[j0:j1,i0:i1])
            sdsn_mean[mind,rind,pind,2] = cf.calc_spatialdsn(t2min[tind,j0:j1,i0:i1],obst2minma[tind,j0:j1,i0:i1],lat2d[j0:j1,i0:i1])
            sdsn_mean[mind,rind,pind,3] = cf.calc_spatialdsn(hfls[tind,j0:j1,i0:i1],obshflsma[tind,j0:j1,i0:i1],lat2d[j0:j1,i0:i1])

            del tind
        
        # Calculate the TX extremes 
        sbias_extr[mind,rind,0:3] = cf.calc_bias_txtn(t2max[:,j0:j1,i0:i1],obst2maxma[:,j0:j1,i0:i1],lat2d[j0:j1,i0:i1],"TX")
        sdsn_extr[mind,rind,0:3] = cf.calc_spatialdsn_txtn(t2max[:,j0:j1,i0:i1],obst2maxma[:,j0:j1,i0:i1],lat2d[j0:j1,i0:i1],"TX")

        # Calculate the TN extremes 
        sbias_extr[mind,rind,3:6] = cf.calc_bias_txtn(t2min[:,j0:j1,i0:i1],obst2minma[:,j0:j1,i0:i1],lat2d[j0:j1,i0:i1],"TN")
        sdsn_extr[mind,rind,3:6] = cf.calc_spatialdsn_txtn(t2min[:,j0:j1,i0:i1],obst2minma[:,j0:j1,i0:i1],lat2d[j0:j1,i0:i1],"TN")

        # Calculate the DTR skill 
        dtr=t2max[:,j0:j1,i0:i1]-t2min[:,j0:j1,i0:i1]
        obsdtr=obst2maxma[:,j0:j1,i0:i1]-obst2minma[:,j0:j1,i0:i1]
        sbias_extr[mind,rind,6] = cf.calc_bias(dtr,obsdtr,lat2d[j0:j1,i0:i1])
        sdsn_extr[mind,rind,6] = cf.calc_spatialdsn(dtr,obsdtr,lat2d[j0:j1,i0:i1])

        # Calculate the PR extremes 
        sbias_extr[mind,rind,7:] = cf.calc_bias_pr(precip[:,j0:j1,i0:i1],obsprecipma[:,j0:j1,i0:i1],lat2d[j0:j1,i0:i1])
        sdsn_extr[mind,rind,7:] = cf.calc_spatialdsn_pr(precip[:,j0:j1,i0:i1],obsprecipma[:,j0:j1,i0:i1],lat2d[j0:j1,i0:i1])

        del bbox,i0,i1,j0,j1
    
    del precip, t2max, t2min, hfls
    del time,timesplit,years,months


/home/561/azh561/Documents/WRF-LIS-CABLE/common_functions.py:42: RuntimeWarning: Mean of empty slice
  omean = np.nanmean(odata,axis=0)
/home/561/azh561/Documents/WRF-LIS-CABLE/common_functions.py:81: RuntimeWarning: Mean of empty slice
  omean = np.nanmean(odata,axis=0)
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.01/lib/python3.6/site-packages/numpy/ma/core.py:6666: RuntimeWarning: overflow encountered in power
  result = np.where(m, fa, umath.power(fa, fb)).view(basetype)
/home/561/azh561/Documents/WRF-LIS-CABLE/common_functions.py:151: RuntimeWarning: Mean of empty slice
  omean = np.nanmean(odata,axis=0)
/home/561/azh561/Documents/WRF-LIS-CABLE/common_functions.py:205: RuntimeWarning: invalid value encountered in greater_equal
  odata = np.ma.masked_array(odata, odata>=1.e20).filled(np.nan)
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.01/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1354: RuntimeWarning: All-NaN slice encountered
  overwrite_input, interp

In [7]:
# Calculate the overall score - climate
s_mean = (sbias_mean + 2*srmse_mean + sphase_mean + sdsn_mean)/5.

# Calculate the overall score - extremes
s_extr = (sbias_extr + sdsn_extr)/2.

# Calculate the Relative Skill Scores

In [9]:
# Determine the relative skill score (Z = x - mu / sigma)

# Climate
s_mean_rel = (s_mean - np.nanmean(s_mean,axis=0)) / np.nanstd(s_mean,axis=0)
sbias_mean_rel = (sbias_mean - np.nanmean(sbias_mean,axis=0)) / np.nanstd(sbias_mean,axis=0)
srmse_mean_rel = (srmse_mean - np.nanmean(srmse_mean,axis=0)) / np.nanstd(srmse_mean,axis=0)
sphase_mean_rel = (sphase_mean - np.nanmean(sphase_mean,axis=0)) / np.nanstd(sphase_mean,axis=0)
sdsn_mean_rel = (sdsn_mean - np.nanmean(sdsn_mean,axis=0)) / np.nanstd(sdsn_mean,axis=0)

# Extremes
s_extr_rel = (s_extr - np.nanmean(s_extr,axis=0)) / np.nanstd(s_extr,axis=0)
sbias_extr_rel = (sbias_extr - np.nanmean(sbias_extr,axis=0)) / np.nanstd(sbias_extr,axis=0)
sdsn_extr_rel = (sdsn_extr - np.nanmean(sdsn_extr,axis=0)) / np.nanstd(sdsn_extr,axis=0)

In [11]:
# Function to create plot
def plot_SKILL(scores,RCMlabels,figurename,vlabels,Rname,mtitle,mx=None,mn=None):
    """This function plots model skill in the context of observational uncertainty.
    
    Input arguments: 
        scores - multi-dimensional array containing the skill scores
        RCMlabels - list containing model/experiment names
        figurename - file name for saving the figure to
        vlabels - array containing the variables included
        Rname - region names
        mtitle - main title
        mx = max value on colorbar
        mn = min value on colorbar
    """

    from matplotlib.colors import BoundaryNorm
    from matplotlib.ticker import MaxNLocator
    from matplotlib.colors import LinearSegmentedColormap

    # Retrieve dimensions
    nmod = scores.shape[0] # Models
    nreg = scores.shape[1] # Regions
    nper = scores.shape[2] # Time periods
    nvar = scores.shape[3] # Variables
        
    # Create figure object and subplots
    plt.rcParams['savefig.dpi'] = 1000
    plt.rcParams["font.weight"] = "bold"
    plt.rcParams["axes.labelweight"] = "bold"
    
    # Create figure object and subplots
    nrow=1
    ncol=2

#    gs = mpl.gridspec.GridSpec(nrow,ncol, width_ratios=[2.,0.1], wspace=0.05)
    gs = mpl.gridspec.GridSpec(nrow,ncol, width_ratios=[2.,0.5], wspace=0.05) # hydro and spin comparison
#    fig = plt.figure(figsize=(16,16))
    fig = plt.figure(figsize=(2,16)) # hydro and spin comparison

    # Make axes
    ax1 = fig.add_subplot(gs[0,0])

    # Colour bar axes (':' as the colour bars cover multiple rows)
    # Use a new subplot so we can control the spacing better
    cgs = mpl.gridspec.GridSpecFromSubplotSpec(1,1, subplot_spec=gs[:,-1], wspace=2)
    cax1 = plt.subplot(cgs[0,0]) 

    if not mx:
        mx=1.
    if not mn:
        mn=0.
    
    # Define the colormap - using a 'traffic light' approach
    nbins=20    
    cmap = plt.get_cmap('RdYlGn')
    cmap.set_bad('w')
    levels = MaxNLocator(nbins=nbins).tick_values(mn,mx)
    norm = BoundaryNorm(levels, ncolors=cmap.N, clip=True)
    
    # Reshape from 4D to 2D array to plot the skill scores as rectanges
    data = np.empty((nreg*nper*nvar,nmod),dtype=np.float64)
    for rr in range(nreg):
        for vv in range(nvar):
            for pp in range(nper):
                data[(rr*nvar*nper) + (vv*nper) + pp,:] = scores[:,rr,pp,vv]

    xarr = np.arange(0.5,nmod+1,1)
    xticks = np.arange(1,nmod+1,1)
    yarr = np.arange(0.5,nreg*nper*nvar+1,1)
    yticks = np.arange(1,nreg*nper*nvar+1,1)
    
    # Plot the skill scores   
    skill = ax1.pcolormesh(xarr,yarr[::-1],data,cmap=cmap,norm=norm)
    ax1.set_yticklabels(vlabels[::-1],ha='right')
    ax1.set_yticks(yticks)
    ax1.set_ylim(yarr[0],yarr[-1])
    ax1.set_xlim(xarr[0],xarr[-1])
    ax1.set_xticklabels(RCMlabels, fontsize = 8, rotation = 90.)
    ax1.set_xticks(xticks)

    for x in range(len(xticks)-1):
        ax1.axvline(x=xticks[x]+0.5, color='black', linestyle='--',linewidth=1.0)

    ax1.axhline(yticks[nvar*nper]-0.5, color='black', linestyle='-',linewidth=1.0)
    ax1.axhline(yticks[2*nvar*nper]-0.5, color='black', linestyle='-',linewidth=1.0)

    # -3 for all, -1 for hydro comparison, -2 for spin comparison
    ax1.text(-2, 53, Rname[0], fontsize=14, rotation = 90.)
    ax1.text(-2, 31, Rname[1], fontsize=14, rotation = 90.)
    ax1.text(-2, 12, Rname[2], fontsize=14, rotation = 90.)
    
    ax1.set_title('%s' %(mtitle), fontweight='bold')
    fig.colorbar(skill,cax1,ticks=levels[::2])
    
    # Finalise the figure layout
    
    fig.tight_layout()   
    fig.subplots_adjust(wspace=0, hspace=0, left=0.25)

    fig.savefig(figurename,bbox_inches = "tight")
    plt.close(fig)


In [12]:
# Function to create plot
def plot_extremes(scores,RCMlabels,figurename,vlabels,Rname,mtitle,mx=None,mn=None):
    """This function plots model skill in the context of observational uncertainty.
    
    Input arguments: 
        scores - multi-dimensional array containing the skill scores
        RCMlabels - list containing model/experiment names
        figurename - file name for saving the figure to
        vlabels - array containing the variables included
        Rname - region names
        mtitle - main title
        mx = max value on colorbar
        mn = min value on colorbar
    """

    from matplotlib.colors import BoundaryNorm
    from matplotlib.ticker import MaxNLocator
    from matplotlib.colors import LinearSegmentedColormap

    # Retrieve dimensions
    nmod = scores.shape[0] # Models
    nreg = scores.shape[1] # Regions
    nind = scores.shape[2] # Extremes indices
        
    # Create figure object and subplots
    plt.rcParams['savefig.dpi'] = 1000
    plt.rcParams["font.weight"] = "bold"
    plt.rcParams["axes.labelweight"] = "bold"
    
    # Create figure object and subplots
    nrow=1
    ncol=2

#    gs = mpl.gridspec.GridSpec(nrow,ncol, width_ratios=[2.,0.1], wspace=0.05)
    gs = mpl.gridspec.GridSpec(nrow,ncol, width_ratios=[2.,0.5], wspace=0.05) # for hydro and spin comparison
#    fig = plt.figure(figsize=(16,16))
    fig = plt.figure(figsize=(2,16)) # for hydro and spin comparison

    # Make axes
    ax1 = fig.add_subplot(gs[0,0])

    # Colour bar axes (':' as the colour bars cover multiple rows)
    # Use a new subplot so we can control the spacing better
    cgs = mpl.gridspec.GridSpecFromSubplotSpec(1,1, subplot_spec=gs[:,-1], wspace=2)
    cax1 = plt.subplot(cgs[0,0]) 

    if not mx:
        mx=1.
    if not mn:
        mn=0.
    
    # Define the colormap - using a 'traffic light' approach
    nbins=20    
    cmap = plt.get_cmap('RdYlGn')
    cmap.set_bad('w')
    levels = MaxNLocator(nbins=nbins).tick_values(mn,mx)
    norm = BoundaryNorm(levels, ncolors=cmap.N, clip=True)
    
    # Reshape from 4D to 2D array to plot the skill scores as rectanges
    data = np.empty((nreg*nind,nmod),dtype=np.float64)
    for rr in range(nreg):
        for ii in range(nind):
            data[(rr*nind) + ii,:] = scores[:,rr,ii]

    xarr = np.arange(0.5,nmod+1,1)
    xticks = np.arange(1,nmod+1,1)
    yarr = np.arange(0.5,nreg*nind+1,1)
    yticks = np.arange(1,nreg*nind+1,1)
    
    # Plot the skill scores   
    skill = ax1.pcolormesh(xarr,yarr[::-1],data,cmap=cmap,norm=norm)
    ax1.set_yticklabels(vlabels[::-1],ha='right')
    ax1.set_yticks(yticks)
    ax1.set_ylim(yarr[0],yarr[-1])
    ax1.set_xlim(xarr[0],xarr[-1])
    ax1.set_xticklabels(RCMlabels, fontsize = 8, rotation = 90.)
    ax1.set_xticks(xticks)

    for x in range(len(xticks)-1):
        ax1.axvline(x=xticks[x]+0.5, color='black', linestyle='--',linewidth=1.0)

    ax1.axhline(yticks[nind]-0.5, color='black', linestyle='-',linewidth=1.0)
    ax1.axhline(yticks[2*nind]-0.5, color='black', linestyle='-',linewidth=1.0)

    # -3 for all, -1 for hydro comparison, -2 for spin comparison
    ax1.text(-2, 29, Rname[0], fontsize=14, rotation = 90.)
    ax1.text(-2, 17, Rname[1], fontsize=14, rotation = 90.)
    ax1.text(-2, 6, Rname[2], fontsize=14, rotation = 90.)
   
    ax1.set_title('%s' %(mtitle), fontweight='bold')
    fig.colorbar(skill,cax1,ticks=levels[::2])
    
    # Finalise the figure layout
    
    fig.tight_layout()   
    fig.subplots_adjust(wspace=0, hspace=0, left=0.25)

    fig.savefig(figurename,bbox_inches = "tight")
    plt.close(fig)


# Plot the Scalar Skill Scores

In [13]:
# Plot the bias skill score
figurename='%s/%s_BIAS.png' %(fig_dir,fig_name_prefix)
plot_SKILL(sbias_mean,rlabels,figurename,vlabels,Rname,'Bias Skill Score')

# Plot the RMSE skill score
figurename='%s/%s_RMSE.png' %(fig_dir,fig_name_prefix)
plot_SKILL(srmse_mean,rlabels,figurename,vlabels,Rname,'RMSE Skill Score')

# Plot the Phase shift skill score
figurename='%s/%s_PHASE.png' %(fig_dir,fig_name_prefix)
plot_SKILL(sphase_mean,rlabels,figurename,vlabels,Rname,'Phase Shift Skill Score')

# Plot the Spatial Distribution skill score
figurename='%s/%s_SDSN.png' %(fig_dir,fig_name_prefix)
plot_SKILL(sdsn_mean,rlabels,figurename,vlabels,Rname,'Spatial Distribution Skill Score')

# Plot the Overall skill score
figurename='%s/%s_ALL.png' %(fig_dir,fig_name_prefix)
plot_SKILL(s_mean,rlabels,figurename,vlabels,Rname,'Overall Skill Score')


# Plot the Relative Skill Scores

In [13]:
# Plot the bias relative skill score
figurename='%s/RELATIVE_%s_BIAS.png' %(fig_dir,fig_name_prefix)
plot_SKILL(sbias_mean_rel,rlabels,figurename,vlabels,Rname,'Bias Relative Skill Score',2.,-2.)

# Plot the RMSE relative skill score
figurename='%s/RELATIVE_%s_RMSE.png' %(fig_dir,fig_name_prefix)
plot_SKILL(srmse_mean_rel,rlabels,figurename,vlabels,Rname,'RMSE Relative Skill Score',2.,-2.)

# Plot the Phase shift relative skill score
figurename='%s/RELATIVE_%s_PHASE.png' %(fig_dir,fig_name_prefix)
plot_SKILL(sphase_mean_rel,rlabels,figurename,vlabels,Rname,'Phase Shift Relative Skill Score',2.,-2.)

# Plot the Spatial Distribution relative skill score
figurename='%s/RELATIVE_%s_SDSN.png' %(fig_dir,fig_name_prefix)
plot_SKILL(sdsn_mean_rel,rlabels,figurename,vlabels,Rname,'Spatial Distribution Relative Skill Score',2.,-2.)

# Plot the overall relative skill score
figurename='%s/RELATIVE_%s_ALL.png' %(fig_dir,fig_name_prefix)
plot_SKILL(s_mean_rel,rlabels,figurename,vlabels,Rname,'Overall Relative Skill Score',2.,-2.)

# Plot the Skill Scores for Extremes Indices

In [14]:
# Plot the bias skill score
figurename='%s/%s_BIAS_EXTREMES.png' %(fig_dir,fig_name_prefix)
plot_extremes(sbias_extr,rlabels,figurename,elabels,Rname,'Bias Skill Score')

# Plot the Spatial Distribution skill score
figurename='%s/%s_SDSN_EXTREMES.png' %(fig_dir,fig_name_prefix)
plot_extremes(sdsn_extr,rlabels,figurename,elabels,Rname,'Spatial Distribution Skill Score')

# Plot the Overall skill score
figurename='%s/%s_ALL_EXTREMES.png' %(fig_dir,fig_name_prefix)
plot_extremes(s_extr,rlabels,figurename,elabels,Rname,'Overall Skill Score')

# Plot the bias skill score
#figurename='%s/RELATIVE_%s_BIAS_EXTREMES.png' %(fig_dir,fig_name_prefix)
#plot_extremes(sbias_extr_rel,rlabels,figurename,elabels,Rname,'Bias Relative Skill Score',2.,-2.)

# Plot the Spatial Distribution skill score
#figurename='%s/RELATIVE_%s_SDSN_EXTREMES.png' %(fig_dir,fig_name_prefix)
#plot_extremes(sdsn_extr_rel,rlabels,figurename,elabels,Rname,'Spatial Distribution Relative Skill Score',2.,-2.)

# Plot the Overall skill score
#figurename='%s/RELATIVE_%s_ALL_EXTREMES.png' %(fig_dir,fig_name_prefix)
#plot_extremes(s_extr_rel,rlabels,figurename,elabels,Rname,'Overall Relative Skill Score',2.,-2.)


# Determine the Top Performers

In [15]:
s_mean_total = np.nansum(s_mean_rel,axis=(2,3))
s_extr_total = np.nansum(s_extr_rel,axis=2)
s_total = s_mean_total + s_extr_total
s_clim_all_total = np.nansum(s_mean_rel,axis=(1,2,3))
s_extr_all_total = np.nansum(s_extr_rel,axis=(1,2))
s_all_total = s_clim_all_total + s_extr_all_total

# For Climate

In [16]:
print('Tropics: \n %s %s \n %s %s \n %s %s \n %s %s \n %s %s' %(
    rlabels[np.argsort(s_mean_total[:,0])[-1]],s_mean_total[np.argsort(s_mean_total[:,0])[-1],0],
    rlabels[np.argsort(s_mean_total[:,0])[-2]],s_mean_total[np.argsort(s_mean_total[:,0])[-2],0],
    rlabels[np.argsort(s_mean_total[:,0])[-3]],s_mean_total[np.argsort(s_mean_total[:,0])[-3],0],
    rlabels[np.argsort(s_mean_total[:,0])[-4]],s_mean_total[np.argsort(s_mean_total[:,0])[-4],0],
    rlabels[np.argsort(s_mean_total[:,0])[-5]],s_mean_total[np.argsort(s_mean_total[:,0])[-5],0]))
print('SEA: \n %s %s \n %s %s \n %s %s \n %s %s \n %s %s' %(
    rlabels[np.argsort(s_mean_total[:,1])[-1]],s_mean_total[np.argsort(s_mean_total[:,1])[-1],1],
    rlabels[np.argsort(s_mean_total[:,1])[-2]],s_mean_total[np.argsort(s_mean_total[:,1])[-2],1],
    rlabels[np.argsort(s_mean_total[:,1])[-3]],s_mean_total[np.argsort(s_mean_total[:,1])[-3],1],
    rlabels[np.argsort(s_mean_total[:,1])[-4]],s_mean_total[np.argsort(s_mean_total[:,1])[-4],1],
    rlabels[np.argsort(s_mean_total[:,1])[-5]],s_mean_total[np.argsort(s_mean_total[:,1])[-5],1]))
print('SWWA: \n %s %s \n %s %s \n %s %s \n %s %s \n %s %s' %(
    rlabels[np.argsort(s_mean_total[:,2])[-1]],s_mean_total[np.argsort(s_mean_total[:,2])[-1],2],
    rlabels[np.argsort(s_mean_total[:,2])[-2]],s_mean_total[np.argsort(s_mean_total[:,2])[-2],2],
    rlabels[np.argsort(s_mean_total[:,2])[-3]],s_mean_total[np.argsort(s_mean_total[:,2])[-3],2],
    rlabels[np.argsort(s_mean_total[:,2])[-4]],s_mean_total[np.argsort(s_mean_total[:,2])[-4],2],
    rlabels[np.argsort(s_mean_total[:,2])[-5]],s_mean_total[np.argsort(s_mean_total[:,2])[-5],2]))
print('All: \n %s %s \n %s %s \n %s %s \n %s %s \n %s %s' %(
    rlabels[np.argsort(s_clim_all_total[:])[-1]],s_clim_all_total[np.argsort(s_clim_all_total[:])[-1]],
    rlabels[np.argsort(s_clim_all_total[:])[-2]],s_clim_all_total[np.argsort(s_clim_all_total[:])[-2]],
    rlabels[np.argsort(s_clim_all_total[:])[-3]],s_clim_all_total[np.argsort(s_clim_all_total[:])[-3]],
    rlabels[np.argsort(s_clim_all_total[:])[-4]],s_clim_all_total[np.argsort(s_clim_all_total[:])[-4]],
    rlabels[np.argsort(s_clim_all_total[:])[-5]],s_clim_all_total[np.argsort(s_clim_all_total[:])[-5]]))


Tropics: 
 RRTMG / MYJ / NTiedtke 17.26430560334481 
 Dudhia / MYJ / NTiedtke 12.71674726679989 
 Dudhia / MYJ / KF 11.941212314907272 
 RRTMG / MYJ / BMJ 11.368750332542003 
 NGoddard / MYJ / NTiedtke 10.49866162093948
SEA: 
 NGoddard / MYJ / KF 11.625198857195826 
 RRTMG / MYNN / NTiedtke 10.689123904484635 
 RRTMG / MYJ / BMJ 10.660921045182793 
 Dudhia / MYJ / BMJ 10.402786477208297 
 RRTMG / MYJ / NTiedtke 9.150365673750542
SWWA: 
 NGoddard / YSU / KF 13.65529706507496 
 Dudhia / MYJ / KF 11.780567309901036 
 NGoddard / YSU / NTiedtke 11.665688284640652 
 NGoddard / MYNN / KF 10.167232347233693 
 RRTMG / MYNN / KF 7.262636475839839
All: 
 RRTMG / MYJ / BMJ 28.199060016871723 
 RRTMG / MYJ / NTiedtke 25.903457236852773 
 NGoddard / YSU / NTiedtke 25.404175257300995 
 RRTMG / MYNN / NTiedtke 24.434166337292393 
 Dudhia / MYJ / KF 22.77603611091817


# For Extremes

In [17]:
print('Tropics: \n %s %s \n %s %s \n %s %s \n %s %s \n %s %s' %(
    rlabels[np.argsort(s_extr_total[:,0])[-1]],s_extr_total[np.argsort(s_extr_total[:,0])[-1],0],
    rlabels[np.argsort(s_extr_total[:,0])[-2]],s_extr_total[np.argsort(s_extr_total[:,0])[-2],0],
    rlabels[np.argsort(s_extr_total[:,0])[-3]],s_extr_total[np.argsort(s_extr_total[:,0])[-3],0],
    rlabels[np.argsort(s_extr_total[:,0])[-4]],s_extr_total[np.argsort(s_extr_total[:,0])[-4],0],
    rlabels[np.argsort(s_extr_total[:,0])[-5]],s_extr_total[np.argsort(s_extr_total[:,0])[-5],0]))
print('SEA: \n %s %s \n %s %s \n %s %s \n %s %s \n %s %s' %(
    rlabels[np.argsort(s_extr_total[:,1])[-1]],s_extr_total[np.argsort(s_extr_total[:,1])[-1],1],
    rlabels[np.argsort(s_extr_total[:,1])[-2]],s_extr_total[np.argsort(s_extr_total[:,1])[-2],1],
    rlabels[np.argsort(s_extr_total[:,1])[-3]],s_extr_total[np.argsort(s_extr_total[:,1])[-3],1],
    rlabels[np.argsort(s_extr_total[:,1])[-4]],s_extr_total[np.argsort(s_extr_total[:,1])[-4],1],
    rlabels[np.argsort(s_extr_total[:,1])[-5]],s_extr_total[np.argsort(s_extr_total[:,1])[-5],1]))
print('SWWA: \n %s %s \n %s %s \n %s %s \n %s %s \n %s %s' %(
    rlabels[np.argsort(s_extr_total[:,2])[-1]],s_extr_total[np.argsort(s_extr_total[:,2])[-1],2],
    rlabels[np.argsort(s_extr_total[:,2])[-2]],s_extr_total[np.argsort(s_extr_total[:,2])[-2],2],
    rlabels[np.argsort(s_extr_total[:,2])[-3]],s_extr_total[np.argsort(s_extr_total[:,2])[-3],2],
    rlabels[np.argsort(s_extr_total[:,2])[-4]],s_extr_total[np.argsort(s_extr_total[:,2])[-4],2],
    rlabels[np.argsort(s_extr_total[:,2])[-5]],s_extr_total[np.argsort(s_extr_total[:,2])[-5],2]))
print('All: \n %s %s \n %s %s \n %s %s \n %s %s \n %s %s' %(
    rlabels[np.argsort(s_extr_all_total[:])[-1]],s_extr_all_total[np.argsort(s_extr_all_total[:])[-1]],
    rlabels[np.argsort(s_extr_all_total[:])[-2]],s_extr_all_total[np.argsort(s_extr_all_total[:])[-2]],
    rlabels[np.argsort(s_extr_all_total[:])[-3]],s_extr_all_total[np.argsort(s_extr_all_total[:])[-3]],
    rlabels[np.argsort(s_extr_all_total[:])[-4]],s_extr_all_total[np.argsort(s_extr_all_total[:])[-4]],
    rlabels[np.argsort(s_extr_all_total[:])[-5]],s_extr_all_total[np.argsort(s_extr_all_total[:])[-5]]))


Tropics: 
 NGoddard / MYNN / KF 9.42778361948729 
 Dudhia / MYNN / KF 8.222146687471142 
 RRTMG / MYJ / NTiedtke 5.841195507643437 
 NGoddard / MYNN / BMJ 5.536750156481055 
 Dudhia / MYNN / BMJ 5.307944379991257
SEA: 
 RRTMG / MYJ / KF 8.139605575006657 
 NGoddard / MYJ / KF 7.6055547739744105 
 RRTMG / MYNN / NTiedtke 7.03715010746593 
 RRTMG / MYNN / KF 6.321298134150096 
 RRTMG / MYNN / GF 5.122490971778326
SWWA: 
 RRTMG / MYNN / KF 4.182523586214824 
 RRTMG / MYJ / KF 3.8696742744988097 
 RRTMG / MYJ / BMJ 3.8553600078129135 
 Dudhia / MYJ / KF 3.803926816549011 
 NGoddard / MYJ / NTiedtke 2.7810970009851577
All: 
 RRTMG / MYNN / KF 15.11919400186291 
 Dudhia / MYJ / KF 13.029289274544585 
 RRTMG / MYNN / NTiedtke 12.420426946844128 
 RRTMG / MYJ / KF 11.575555499710148 
 RRTMG / MYNN / GF 11.398660678552119


# For Total

In [18]:
print('Tropics: \n %s %s \n %s %s \n %s %s \n %s %s \n %s %s' %(
    rlabels[np.argsort(s_total[:,0])[-1]],s_total[np.argsort(s_total[:,0])[-1],0],
    rlabels[np.argsort(s_total[:,0])[-2]],s_total[np.argsort(s_total[:,0])[-2],0],
    rlabels[np.argsort(s_total[:,0])[-3]],s_total[np.argsort(s_total[:,0])[-3],0],
    rlabels[np.argsort(s_total[:,0])[-4]],s_total[np.argsort(s_total[:,0])[-4],0],
    rlabels[np.argsort(s_total[:,0])[-5]],s_total[np.argsort(s_total[:,0])[-5],0]))
print('SEA: \n %s %s \n %s %s \n %s %s \n %s %s \n %s %s' %(
    rlabels[np.argsort(s_total[:,1])[-1]],s_total[np.argsort(s_total[:,1])[-1],1],
    rlabels[np.argsort(s_total[:,1])[-2]],s_total[np.argsort(s_total[:,1])[-2],1],
    rlabels[np.argsort(s_total[:,1])[-3]],s_total[np.argsort(s_total[:,1])[-3],1],
    rlabels[np.argsort(s_total[:,1])[-4]],s_total[np.argsort(s_total[:,1])[-4],1],
    rlabels[np.argsort(s_total[:,1])[-5]],s_total[np.argsort(s_total[:,1])[-5],1]))
print('SWWA: \n %s %s \n %s %s \n %s %s \n %s %s \n %s %s' %(
    rlabels[np.argsort(s_total[:,2])[-1]],s_total[np.argsort(s_total[:,2])[-1],2],
    rlabels[np.argsort(s_total[:,2])[-2]],s_total[np.argsort(s_total[:,2])[-2],2],
    rlabels[np.argsort(s_total[:,2])[-3]],s_total[np.argsort(s_total[:,2])[-3],2],
    rlabels[np.argsort(s_total[:,2])[-4]],s_total[np.argsort(s_total[:,2])[-4],2],
    rlabels[np.argsort(s_total[:,2])[-5]],s_total[np.argsort(s_total[:,2])[-5],2]))
print('All: \n %s %s \n %s %s \n %s %s \n %s %s \n %s %s' %(
    rlabels[np.argsort(s_all_total[:])[-1]],s_all_total[np.argsort(s_all_total[:])[-1]],
    rlabels[np.argsort(s_all_total[:])[-2]],s_all_total[np.argsort(s_all_total[:])[-2]],
    rlabels[np.argsort(s_all_total[:])[-3]],s_all_total[np.argsort(s_all_total[:])[-3]],
    rlabels[np.argsort(s_all_total[:])[-4]],s_all_total[np.argsort(s_all_total[:])[-4]],
    rlabels[np.argsort(s_all_total[:])[-5]],s_all_total[np.argsort(s_all_total[:])[-5]]))


Tropics: 
 RRTMG / MYJ / NTiedtke 23.105501110988246 
 NGoddard / MYNN / KF 18.78660694737995 
 Dudhia / MYJ / KF 17.229307949115835 
 Dudhia / MYNN / KF 16.871589556267438 
 RRTMG / MYJ / BMJ 14.402003416741627
SEA: 
 NGoddard / MYJ / KF 19.230753631170238 
 RRTMG / MYNN / NTiedtke 17.726274011950565 
 RRTMG / MYJ / BMJ 15.097415764852135 
 RRTMG / MYJ / NTiedtke 13.00628972334271 
 RRTMG / MYNN / GF 11.712486787869373
SWWA: 
 Dudhia / MYJ / KF 15.584494126450046 
 NGoddard / YSU / KF 15.360001553098781 
 RRTMG / MYNN / KF 11.445160062054663 
 RRTMG / MYJ / BMJ 10.024748646959846 
 RRTMG / MYJ / KF 9.228612221419832
All: 
 RRTMG / MYJ / BMJ 39.5241678285536 
 RRTMG / MYNN / NTiedtke 36.85459328413652 
 RRTMG / MYJ / NTiedtke 36.537992450500454 
 Dudhia / MYJ / KF 35.80532538546275 
 NGoddard / MYNN / KF 30.142887908340654
